# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np

from sqlalchemy import create_engine


import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier


[nltk_data] Downloading package punkt to /home/cry/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/cry/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/cry/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /home/cry/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/cry/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
# load data from database
engine = create_engine('sqlite:///etl_pipeline.db')
df = pd.read_sql_table('etl_pipeline', engine)  
X = df.message.head(500)
Y = df.iloc[:, 4:].head(500)

In [22]:
X.sample(2)

388    GOOD DAY! SWEETHEART I LOVE YOU FOREVER BABY
270       rin more or less 3000 people - incomplete
Name: message, dtype: object

In [23]:
Y.sample(5)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
187,1,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
295,1,1,0,1,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,1,1
431,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
298,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
169,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [24]:
def tokenize(text):
    '''
    Function: Process the text to return a standard version 
    Params:
      text(str): the message
    Return:
      lemmed(list of str): a lis of the tokenized words
    '''
    tokens = word_tokenize(text)
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = text.split()
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    # Lemmatize verbs by specifying pos
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]

    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [25]:
pipeline = Pipeline([
        ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

#MultiOutputClassifier(KNeighborsClassifier())

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
# train classifier
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('text_pipeline',
                 Pipeline(steps=[('vect',
                                  CountVectorizer(tokenizer=<function tokenize at 0x7f95defc4820>)),
                                 ('tfidf', TfidfTransformer())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
def display_results(model, X_test,Y_test):
    '''
    Report the f1 score, precision and recall of the model
    Params : model, test set(X_test and Y_test)
    Output: Report the f1 score, precision and recall for each output category of the dataset
    '''
    Y_pred = model.predict(X_test)
    for i, col in enumerate(Y_test):
        print(col)
        print(classification_report(Y_test[col], Y_pred[:, i]))

In [28]:
display_results(pipeline, X_test, Y_test)

related
              precision    recall  f1-score   support

           0       0.50      0.17      0.26        23
           1       0.84      0.96      0.89       102

    accuracy                           0.82       125
   macro avg       0.67      0.57      0.58       125
weighted avg       0.78      0.82      0.78       125

request
              precision    recall  f1-score   support

           0       0.72      0.85      0.78        59
           1       0.84      0.71      0.77        66

    accuracy                           0.78       125
   macro avg       0.78      0.78      0.78       125
weighted avg       0.79      0.78      0.78       125

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       124
           1       0.00      0.00      0.00         1

    accuracy                           0.99       125
   macro avg       0.50      0.50      0.50       125
weighted avg       0.98      0.99      0.99       12

/home/cry/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [32]:
parameters = {
        'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'text_pipeline__vect__max_features': (None, 5000, 10000),
        'text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__min_samples_split': [2, 4]
    }


# parameters =  {'text_pipeline__tfidf__use_idf': (True, False), 
#               'clf__estimator__n_estimators': [50, 100], 
#               'clf__estimator__min_samples_split': [2, 4]} 


cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [33]:
cv.fit(X_train, Y_train)
display_results(cv, X_test, Y_test)

related
              precision    recall  f1-score   support

           0       0.62      0.22      0.32        23
           1       0.85      0.97      0.90       102

    accuracy                           0.83       125
   macro avg       0.74      0.59      0.61       125
weighted avg       0.81      0.83      0.80       125

request
              precision    recall  f1-score   support

           0       0.76      0.85      0.80        59
           1       0.85      0.76      0.80        66

    accuracy                           0.80       125
   macro avg       0.80      0.80      0.80       125
weighted avg       0.81      0.80      0.80       125

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       124
           1       0.00      0.00      0.00         1

    accuracy                           0.99       125
   macro avg       0.50      0.50      0.50       125
weighted avg       0.98      0.99      0.99       12

/home/cry/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv_optimized, open('disaster_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.